# Scaling-Up Neural Networks 

Now that we are familiar with sequence to sequence models let’s discuss  how to scale them to real world problems. In the previous example we used a very small vocabulary (4 symbols $“a,b,c,d”$)  but when dealing with tasks such as automatic machine translation our vocabulary can contain thousands of unique words, that means that the last layer of our network must be the dimension as the size of the vocabulary and $softmax$ must be applied on an extremely long vector. When training a network we cannot afford such a costly operation.

That's why hierarchical softmax was invented, it can increase softmax time by up to $log(n)$ without adding compromising too much of the network reliability.

# The Softmax Function

The softmax function, or normalized exponential is used to transform a $n$ dimensional vector into a probabilty over $n$ classes, formaly it is defined as: $P(class=i | x) = \dfrac{e^{x_i}}{\sum_{j=1}^ne^{x_j}}$.

NumPy code for SoftMax:

In [15]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In order to calculate the softmax we need to $O(n)$ time. Let's see how can we reduce it to $O(log(n))$.

# The Hierarchical Softmax Function

The hierarchical softmax function first arranges the output vocabulary in an hierarchical tree (we will discuss later how this tree can be constructed). Once we constructed the tree we can use softmax to select each branch of the tree. For example, given the following hierarchy and the value of the last hidden layer is $V$:

<img src="img/HierSoftMax.jpg" width="25%" height="25%">

The probabilty of the network to predict "D" is (the probabilty of choosing the 2nd branch in $B_1$)*(the provavilty of choosing the 1st branch on $B_3$). More formally:  $P(class=D | V) = softmax(V \times W_{B_1})_1*softmax(V \times W_{B_3})_0$

Now that we know what is hierarchical softmax we can start implementing it. We will create an hier_softmax that will recieve an output hierarchy and will be able to calculate of the probabilty of an output given $V$ and to generate to most likely output given a vector $V$.

First we will define some auxiliary functions to handle trees:

In [16]:
from random import shuffle
from copy import copy

def _get_subtrees(tree):
    yield tree
    for subtree in tree:
        if type(subtree) == list:
            for x in _get_subtrees(subtree):
                yield x


# Returns pairs of paths and leafves of a tree
def _get_leaves_paths(tree):
    for i, subtree in enumerate(tree):
        if type(subtree) == list:
            for path, value in _get_leaves_paths(subtree):
                yield [i] + path, value
        else:
            yield [i], subtree


# Returns the number of nodes in a tree (not including root)
def _count_nodes(tree):
    size = 0
    for node in tree:
        if type(node) == list:
            size += 1 + _count_nodes(node)
    return size


# Returns all the nodes in a path
def _get_nodes(tree, path):
    next_node = 0
    nodes = []
    for decision in path:
        nodes.append(next_node)
        next_node += 1 + _count_nodes(tree[:decision])
        tree = tree[decision]
    return nodes


# turns a list to a binary tree
def random_binary_full_tree(outputs):
    outputs = copy(outputs)
    shuffle(outputs)

    while len(outputs) > 2:
        temp_outputs = []
        for i in range(0, len(outputs), 2):
            if len(outputs) - (i+1) > 0:
                temp_outputs.append([outputs[i], outputs[i+1]])
            else:
                temp_outputs.append(outputs[i])
        outputs = temp_outputs
    return outputs

Let's test the auxiliary functions:

In [17]:
tree = random_binary_full_tree(range(10))
print 'Our tree:',tree

print 'All subtrees:'
for subtree in _get_subtrees(tree):
    print '\t',subtree

print 'All paths and leaves:'
for subtree in _get_leaves_paths(tree):
    print '\t',subtree
    
print 'Number of nodes in the tree:',_count_nodes(tree)

print 'all nodes in path [0, 0, 0, 0]:'
for nodes in _get_nodes(tree, [0, 0, 0, 0]):
    print '\t',nodes


Our tree: [[[[7, 8], [6, 5]], [[4, 1], [0, 2]]], [9, 3]]
All subtrees:
	[[[[7, 8], [6, 5]], [[4, 1], [0, 2]]], [9, 3]]
	[[[7, 8], [6, 5]], [[4, 1], [0, 2]]]
	[[7, 8], [6, 5]]
	[7, 8]
	[6, 5]
	[[4, 1], [0, 2]]
	[4, 1]
	[0, 2]
	[9, 3]
All paths and leaves:
	([0, 0, 0, 0], 7)
	([0, 0, 0, 1], 8)
	([0, 0, 1, 0], 6)
	([0, 0, 1, 1], 5)
	([0, 1, 0, 0], 4)
	([0, 1, 0, 1], 1)
	([0, 1, 1, 0], 0)
	([0, 1, 1, 1], 2)
	([1, 0], 9)
	([1, 1], 3)
Number of nodes in the tree: 8
all nodes in path [0, 0, 0, 0]:
	0
	1
	2
	3


The hier softmax class:

In [18]:
import pycnn as pc

class hier_softmax:
    def __init__(self, tree, contex_size, model):
        #create a weight matrix and bias vector for each node in the tree
        for i, subtree in enumerate(_get_subtrees(tree)):
            model.add_parameters("softmax_node_"+str(i)+"_w", (len(subtree), contex_size))
            model.add_parameters("softmax_node_" + str(i) + "_b", len(subtree))
        
        #create a dictionary from each value to its path
        value_to_path_and_nodes_dict = {}
        for path, value in _get_leaves_paths(tree):
            nodes = _get_nodes(tree, path)
            value_to_path_and_nodes_dict[char2int[value]] = path, nodes
        self.value_to_path_and_nodes_dict = value_to_path_and_nodes_dict
        self.model = model
        self.tree = tree
    
    #get the loss on a given value (for training)
    def get_loss(self, context, value):
        loss = []
        path, nodes = self.value_to_path_and_nodes_dict[value]
        for p, n in zip(path, nodes):
            w = pc.parameter(self.model["softmax_node_"+str(n)+"_w"])
            b = pc.parameter(self.model["softmax_node_" + str(n) + "_b"])
            probs = pc.softmax(w*context+b)
            loss.append(-pc.log(pc.pick(probs, p)))
        return pc.esum(loss)

    #get the most likely
    def generate(self, context):
        best_value = None
        best_loss = float(100000)
        for value in self.value_to_path_and_nodes_dict:
            loss = self.get_loss(context, value)
            if loss < best_loss:
                best_loss = loss
                best_value = value
        return best_value

Now we can test the performance improvement we can get from the hier_softmax. Again, we will learn the reverse function, but this time on a big vocabulary

In [19]:
from random import choice, randrange

EOS = "<EOS>" #all strings will end with the End Of String token
characters = [str(i) for i in range(1000)]
characters.append(EOS)

int2char = list(characters)
char2int = {c:i for i,c in enumerate(characters)}

VOCAB_SIZE = len(characters)

def sample_model(min_length, max_lenth):
    random_length = randrange(min_length, max_lenth)                             # Pick a random length
    random_char_list = [choice(characters[:-1]) for _ in xrange(random_length)]  # Pick random chars
    random_string = ' '.join(random_char_list)
    reverse_string = ' '.join(random_char_list[::-1])
    return random_string, reverse_string  # Return the random string and its reverse

print sample_model(4, 5)

('684 437 885 280', '280 885 437 684')


In [20]:
import matplotlib.pyplot as plt
from datetime import datetime
MAX_STRING_LEN = 5

train_set = [sample_model(1, MAX_STRING_LEN) for _ in xrange(3000)]
val_set = [sample_model(1, MAX_STRING_LEN) for _ in xrange(50)]

def train(network, train_set, val_set, epochs = 20):
    def get_val_set_loss(network, val_set):
        loss = [network.get_loss(input_string, output_string).value() for input_string, output_string in val_set]
        return sum(loss)
    
    train_set = train_set*epochs
    trainer = pc.SimpleSGDTrainer(network.model)
    start = datetime.now()
    for i, training_example in enumerate(train_set):
        input_string, output_string = training_example
        
        loss = network.get_loss(input_string, output_string)
        loss_value = loss.value()
        loss.backward()
        trainer.update()
    print datetime.now()-start
        

    print 'loss on validation set:', get_val_set_loss(network, val_set)

Now that we have a large vocab data we can measure the training time of the attention model

In [21]:
from models import AttentionNetwork

ENC_RNN_NUM_OF_LAYERS = 1
DEC_RNN_NUM_OF_LAYERS = 1
EMBEDDINGS_SIZE = 200
ENC_STATE_SIZE = 210
DEC_STATE_SIZE = 210

att = AttentionNetwork(ENC_RNN_NUM_OF_LAYERS, DEC_RNN_NUM_OF_LAYERS, EMBEDDINGS_SIZE, ENC_STATE_SIZE, DEC_STATE_SIZE)

In [22]:
train(att, train_set, val_set)

0:08:33.284177
loss on validation set: 3.17660824512


In [23]:
output_tree = random_binary_full_tree(characters)


RNN_BUILDER = pc.LSTMBuilder
class AttentionNetworkWithHierSoftmax(AttentionNetwork):
    def __init__(self, enc_layers, dec_layers, embeddings_size, enc_state_size, dec_state_size, tree):
        self.model = pc.Model()

        # the embedding paramaters
        self.model.add_lookup_parameters("lookup", (VOCAB_SIZE, embeddings_size))

        # the rnns
        self.ENC_RNN = RNN_BUILDER(enc_layers, embeddings_size, enc_state_size, self.model)
        self.DEC_RNN = RNN_BUILDER(dec_layers, enc_state_size, dec_state_size, self.model)
        
        # attention weights
        self.model.add_parameters("attention_w1", (enc_state_size, enc_state_size))
        self.model.add_parameters("attention_w2", (enc_state_size, dec_state_size))
        self.model.add_parameters("attention_v", (1, enc_state_size))

        self.enc_state_size = enc_state_size
        
        self.tree = tree
        self.hier_softmax = hier_softmax(tree, dec_state_size, self.model)
    
    def get_loss(self, input_string, output_string):
        input_string = self._add_eos(input_string)
        output_string = self._add_eos(output_string)

        pc.renew_cg()

        embedded_string = self._embed_string(input_string)
        encoded_string = self._encode_string(embedded_string)

        rnn_state = self.DEC_RNN.initial_state().add_input(pc.vecInput(self.enc_state_size))

        loss = []
        for output_char in output_string:
            attended_encoding = self._attend(encoded_string, rnn_state)
            rnn_state = rnn_state.add_input(attended_encoding)
            loss.append(self.hier_softmax.get_loss(rnn_state.output(), output_char))
        loss = pc.esum(loss)
        return loss
        
    
    def generate(self, input_string):
        input_string = self._add_eos(input_string)

        pc.renew_cg()

        embedded_string = self._embed_string(input_string)
        encoded_string = self._encode_string(embedded_string)

        rnn_state = self.DEC_RNN.initial_state().add_input(pc.vecInput(self.enc_state_size))

        output_string = []
        while True:
            attended_encoding = self._attend(encoded_string, rnn_state)
            rnn_state = rnn_state.add_input(attended_encoding)
            predicted_char = self.self.hier_softmax.generate(rnn_state.output())
            output_string.append(predicted_char)
            if predicted_char == EOS or len(output_string) > 2*len(input_string):
                break
        output_string = ' '.join(output_string)
        return output_string.replace('<EOS>', '')        

In [24]:
att = AttentionNetworkWithHierSoftmax(
    ENC_RNN_NUM_OF_LAYERS, DEC_RNN_NUM_OF_LAYERS, EMBEDDINGS_SIZE, ENC_STATE_SIZE, DEC_STATE_SIZE, output_tree)

train(att, train_set, val_set)

0:07:28.167242
loss on validation set: 1.70575599489
